In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from Affine_Transformations import generate_strain_tensors

In [3]:
# Load the dataset
file_path = "data/DeDoDe_Descriptors_Dataset.pth"  # Change this to your actual path
data = torch.load(file_path)
all_base_descriptors = data['descriptors']# Assuming these keys exist
all_transformed_descriptors = data['deformed_descriptors']

/tmp/ipykernel_117096/61485861.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)


In [4]:
# Select the data corresponding to transformation x
transformation = 37
transformations = data['transformations']
transformations = torch.repeat_interleave(transformations, repeats=11, dim=1).flatten()
idx = transformations == transformation
base_descriptors = all_base_descriptors[idx]
transformed_descriptors = all_transformed_descriptors[idx]
print(f'Transformation Tensor: {generate_strain_tensors()[transformation]}')
print(f'Number of Descriptions: {len(base_descriptors)}')
print(f'Mean Cosine Similarity: {torch.nn.functional.cosine_similarity(base_descriptors,transformed_descriptors,dim = 1).mean()}')

Transformation Tensor: (-0.25, 0.0, 0.0)
Number of Descriptions: 1023
Mean Cosine Similarity: 0.8963428496344185


In [5]:
# Create a dataset and dataloader
dataset = TensorDataset(base_descriptors, transformed_descriptors)
# dataset = TensorDataset(all_base_descriptors, all_transformed_descriptors)
train_size = 0.8
val_size = 0.2

train_dataset, val_dataset = random_split(dataset,[train_size,val_size])

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last = False)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, drop_last = False)

In [6]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim,output_dim)
            
            # nn.Linear(input_dim, hidden_dim),
            # # nn.BatchNorm1d(hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim, output_dim)
            
            # nn.Linear(input_dim, hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim, hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.model(x)

In [7]:
# Define residual MLP model
class Residual_MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256, num_layers = 1):
        super(Residual_MLP, self).__init__()
        
        layers = []

        if num_layers == 0:
            layers.append(nn.Linear(input_dim, output_dim))
        else:
            # First hidden layer
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())

            for _ in range(num_layers - 1):
                layers.append(nn.Linear(hidden_dim, hidden_dim))
                layers.append(nn.ReLU())

            layers.append(nn.Linear(hidden_dim, output_dim))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        residual = self.model(x)
        return x + residual

In [8]:
# Define residual conditional MLP


In [9]:
# Initialize the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
input_dim = base_descriptors.shape[1]
output_dim = transformed_descriptors.shape[1]
# model = MLP(input_dim,output_dim).double().to(device)
model = Residual_MLP(input_dim,output_dim,hidden_dim=256,num_layers = 4).double().to(device)

cuda


In [10]:
# Potentially apply identity initialisation to model
identity_initialisation = True

def init_identity(m):
    if isinstance(m, nn.Linear):
        nn.init.eye_(m.weight)  # Initialize as identity
        if m.bias is not None:
            nn.init.zeros_(m.bias)

if identity_initialisation and isinstance(model,MLP):
    model.apply(init_identity)
    print('identity applied')


In [12]:
# Define loss function and optimizer
def cosine_similarity(output,target,reduction = 'mean'):
    loss = 1 - torch.nn.functional.cosine_similarity(output,target)

    if reduction == 'mean':
        return loss.mean()
    elif reduction == 'none':
        return loss

# criterion = nn.CosineEmbeddingLoss(reduction='none')
# criterion = nn.MSELoss(reduction='none')
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

In [13]:
# One eval round before training
model.eval()
start_losses = []
with torch.no_grad():
    for base, transformed in val_dataloader:
        base, transformed = base.to(device), transformed.to(device)
        output = model(base)
        # losses = criterion(output,transformed)
        losses = cosine_similarity(output,transformed,'none')
        loss = losses.mean()
        start_losses.extend(losses.cpu().numpy())

start_loss = np.mean(start_losses)
print(f'Loss before training: {start_loss}')

writer = SummaryWriter(log_dir="runs/tain_one_transformation")

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    train_losses = []
    for base, transformed in train_dataloader:
        base, transformed = base.to(device), transformed.to(device)
        
        optimizer.zero_grad()
        output = model(base)
        # losses = criterion(output, transformed)
        losses = cosine_similarity(output,transformed,'none')
        loss = losses.mean()
        loss.backward()
        optimizer.step()
        
        train_losses.extend(losses.cpu().detach().numpy())

    model.eval()
    val_losses = []
    val_cosines = []
    with torch.no_grad():
        for base, transformed in val_dataloader:
            base, transformed = base.to(device), transformed.to(device)
            output = model(base)
            # losses = criterion(output,transformed)
            losses = cosine_similarity(output,transformed,'none')
            cosines = 1-losses
            val_losses.extend(losses.cpu().numpy())
            val_cosines.extend(cosines.cpu().numpy())

    avg_train_loss = np.mean(train_losses)
    std_train_loss = np.std(train_losses)
    
    avg_val_loss = np.mean(val_losses)
    std_val_loss = np.std(val_losses)

    avg_val_cos = np.mean(val_cosines)
    std_val_cos = np.mean(val_cosines)

    writer.add_scalar("Loss/Train", avg_train_loss, epoch)
    writer.add_scalar("Loss_std/Train", std_train_loss, epoch)
    writer.add_scalar("Loss/Validation", avg_val_loss, epoch)
    writer.add_scalar("Loss_std/Validation", std_val_loss, epoch)

    scheduler.step(avg_val_loss)

    if (epoch+1)%5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}, Val Cos: {avg_val_cos:.6f}")

Loss before training: 0.10323427319571947
Epoch 5/100, Train Loss: 0.103395, Val Loss: 0.097958, Val Cos: 0.902042
Epoch 10/100, Train Loss: 0.100405, Val Loss: 0.096314, Val Cos: 0.903686
Epoch 15/100, Train Loss: 0.094292, Val Loss: 0.093001, Val Cos: 0.906999
Epoch 20/100, Train Loss: 0.087186, Val Loss: 0.089705, Val Cos: 0.910295
Epoch 25/100, Train Loss: 0.081144, Val Loss: 0.087298, Val Cos: 0.912702
Epoch 30/100, Train Loss: 0.076081, Val Loss: 0.085452, Val Cos: 0.914548
Epoch 35/100, Train Loss: 0.071985, Val Loss: 0.083884, Val Cos: 0.916116
Epoch 40/100, Train Loss: 0.068559, Val Loss: 0.082601, Val Cos: 0.917399
Epoch 45/100, Train Loss: 0.065652, Val Loss: 0.081506, Val Cos: 0.918494
Epoch 50/100, Train Loss: 0.063016, Val Loss: 0.080563, Val Cos: 0.919437
Epoch 55/100, Train Loss: 0.060671, Val Loss: 0.079747, Val Cos: 0.920253
Epoch 60/100, Train Loss: 0.058664, Val Loss: 0.079057, Val Cos: 0.920943
Epoch 65/100, Train Loss: 0.056634, Val Loss: 0.078530, Val Cos: 0.9214

In [28]:
# Save the trained model
torch.save(model.state_dict(), "models/single_transformation_model.pth")
print("Model training complete and saved.")

Model training complete and saved.
